In [35]:
import numpy as np
from imageio import imread
import os
from pathlib import Path

In [ ]:
BASE_DIR = Path("/ofo-share/repos-david/UCNRS-experiments/data/labels/geograypher_rendered_fixed")

folder_sizes = {}

In [ ]:
for parent_folder in BASE_DIR.iterdir():
    image_count = 0
    image_folder = parent_folder / "images"

    for subfolder in image_folder.rglob("*"):
        if subfolder.is_file():
            image_count += 1

    folder_sizes[parent_folder.name] = image_count

In [13]:
folder_sizes

{'000568': 47,
 '000549': 306,
 '000565': 48,
 '000555': 166,
 '000610': 197,
 '000611': 163,
 '000557': 441,
 '000559': 596,
 '000908': 595,
 '000479': 269,
 '000911': 303,
 '000912': 117,
 '000548': 90,
 '000544': 44,
 '000563': 116,
 '000575': 0,
 '000551': 76,
 '000578': 729,
 '000574': 45,
 '000612': 246,
 '000614': 1021,
 '000564': 132,
 '000913': 1124,
 '000567': 99,
 '000613': 261,
 '000576': 147,
 '000577': 122}

In [17]:
sum(folder_sizes.values())

7500

In [ ]:
0.2*7500  # 20% of 7500 for val set

1500.0

In [ ]:
# Sort folders based on size
sorted_folders = sorted(folder_sizes.items(), key=lambda x: x[1])
sorted_folders

[('000575', 0),
 ('000544', 44),
 ('000574', 45),
 ('000568', 47),
 ('000565', 48),
 ('000551', 76),
 ('000548', 90),
 ('000567', 99),
 ('000563', 116),
 ('000912', 117),
 ('000577', 122),
 ('000564', 132),
 ('000576', 147),
 ('000611', 163),
 ('000555', 166),
 ('000610', 197),
 ('000612', 246),
 ('000613', 261),
 ('000479', 269),
 ('000911', 303),
 ('000549', 306),
 ('000557', 441),
 ('000908', 595),
 ('000559', 596),
 ('000578', 729),
 ('000614', 1021),
 ('000913', 1124)]

In [ ]:
del sorted_folders[0]  # folder 000575 has 0 images, so delete it

In [33]:
# Select smallest folders until sum reaches 1500
val_set = []
train_set = []
val_count = 0
VAL_TARGET = 1500

for folder, count in sorted_folders:
    if val_count + count <= VAL_TARGET:
        val_set.append(folder)
        val_count += count
    else:
        train_set.append(folder)

print(f"Val set: {val_set} \nTrain set: {train_set}")

Val set: ['000544', '000574', '000568', '000565', '000551', '000548', '000567', '000563', '000912', '000577', '000564', '000576', '000611', '000555'] 
Train set: ['000610', '000612', '000613', '000479', '000911', '000549', '000557', '000908', '000559', '000578', '000614', '000913']


In [27]:
BASE_DIR = Path("/ofo-share/repos-david/UCNRS-experiments/data/labels/geograypher_rendered_fixed")
OUTPUT_DIR = Path("/ofo-share/scratch-amritha/NRS-data/geograypher_rendered/80_20_folder_level_split")
IMG_DIR = OUTPUT_DIR / "img_dir"
ANN_DIR = OUTPUT_DIR / "ann_dir"

TRAIN_DIR = "train"
VAL_DIR = "val"

for d in [IMG_DIR / TRAIN_DIR, IMG_DIR / VAL_DIR, ANN_DIR / TRAIN_DIR, ANN_DIR / VAL_DIR]:
    d.mkdir(parents=True, exist_ok=True)

In [36]:
for folder in train_set + val_set:
    
    if folder in val_set:
        img_target_dir = IMG_DIR / VAL_DIR
        ann_target_dir = ANN_DIR / VAL_DIR
    else:
        img_target_dir = IMG_DIR / TRAIN_DIR
        ann_target_dir = ANN_DIR / TRAIN_DIR

    image_folder = BASE_DIR / folder / "images"
    label_folder = BASE_DIR / folder / "renders"

    for subfolder in image_folder.iterdir():
        label_subdir = label_folder / subfolder.name

        for sub_subfolder in subfolder.iterdir():
            label_sub_subfolder = label_subdir / sub_subfolder.name

            if not label_sub_subfolder.exists(): # Skip if it doesn't exist
                continue

            for img_path in sub_subfolder.iterdir():
                label_path = label_sub_subfolder / (img_path.stem + ".png")

                # Ignore images with no annotations
                if np.all(imread(label_path) == 255):
                    continue

                image_save_name = img_path.stem + "_rgb.JPG"
                label_save_name = label_path.stem + "_segmentation.png"
                
                os.symlink(img_path, img_target_dir / image_save_name)
                os.symlink(label_path, ann_target_dir / label_save_name)



/tmp/ipykernel_6539/4223958575.py:26: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  if np.all(imread(label_path) == 255):
